In [1]:
!pip install -q ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 137.0 MB/s eta 0:00:00


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="0veS42farWpmazy4gGre")
project = rf.workspace("janith-dtugm").project("cucumber-leaf-detection")
version = project.version(2)
dataset = version.download("yolov8")
print("Dataset path:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cucumber-Leaf-detection-2 in yolov8:: 100%|██████████| 4816/4816 [00:00<00:00, 5951.53it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset path: /content/cucumber-Leaf-detection-2


In [3]:
import os

data_yaml = os.path.join(dataset.location, "data.yaml")
print("data.yaml path:", data_yaml)

!cat {data_yaml}

data.yaml path: /content/cucumber-Leaf-detection-2/data.yaml
names:
- Cucumber Leaf
nc: 1
roboflow:
  license: CC BY 4.0
  project: cucumber-leaf-detection
  url: https://universe.roboflow.com/janith-dtugm/cucumber-leaf-detection/dataset/2
  version: 2
  workspace: janith-dtugm
test: ../test/images
train: ../train/images
val: ../valid/images


In [4]:
import yaml

data_yaml = "/content/cucumber-Leaf-detection-2/data.yaml"

with open(data_yaml, "r") as f:
    data = yaml.safe_load(f)

data["names"] = ["cucumber_leaf"]
data["nc"] = 1

with open(data_yaml, "w") as f:
    yaml.safe_dump(data, f)

!cat {data_yaml}

names:
- cucumber_leaf
nc: 1
roboflow:
  license: CC BY 4.0
  project: cucumber-leaf-detection
  url: https://universe.roboflow.com/janith-dtugm/cucumber-leaf-detection/dataset/2
  version: 2
  workspace: janith-dtugm
test: ../test/images
train: ../train/images
val: ../valid/images


In [11]:
!yolo detect train data=/content/cucumber-Leaf-detection-2/data.yaml model=yolov8n.pt epochs=30 imgsz=640 batch=16

Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/cucumber-Leaf-detection-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

In [12]:
from ultralytics import YOLO

WEIGHTS_PATH = "/content/runs/detect/train4/weights/best.pt"
DATA_YAML_PATH = "/content/cucumber-Leaf-detection-2/data.yaml"   # must point to your val/test set

model = YOLO(WEIGHTS_PATH)

metrics = model.val(
    data=DATA_YAML_PATH,
    imgsz=640,
    split="val",     # or "test"
    plots=True,      # confusion matrix + PR curves
    save_json=True
)

print(metrics.results_dict)     # full summary dict :contentReference[oaicite:1]{index=1}
print("mAP50-95:", metrics.box.map)
print("mAP50:", metrics.box.map50)
print("Precision:", metrics.box.mp)
print("Recall:", metrics.box.mr)

P, R = float(metrics.box.mp), float(metrics.box.mr)
print("F1 (from mean P,R):", (2*P*R/(P+R)) if (P+R) else 0.0)
print("Saved plots to:", metrics.save_dir)


Ultralytics 8.3.246 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1795.4±518.4 MB/s, size: 71.1 KB)
val: Scanning /content/cucumber-Leaf-detection-2/valid/labels.cache... 201 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 201/201 391.9Kit/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 164, len(boxes) = 200. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 3.8it/s 3.4s
                   all        201        200      0.968       0.98      0.989      0.922
Speed: 2.6ms preprocess, 4.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving /content/runs/detect/val2